# Twitter Data Collection

We want to collect the data from Twitter according to given screen names. The data include a lot of characters that are not related us such as emoji and urls. Therefore in this notebook, we will clean them into all tweets.

## Getting Twitter API keys:

* Login to https://apps.twitter.com and "Create a New App".
* Fill the form.
* Click "Keys and Access Tokens" tab and copy "API key", API secret".
* Click "Create my Access Token" and copy "Access token" and "Access token secret".

#### Requirements:
* tweepy


In [1]:
import tweepy #Twitter API helper package
from tweepy import OAuthHandler
import re #Regex

consumer_key = "utTM4qfuhmzeLUxRkBb1xb12P" # API key
consumer_secret = "XteCQjAZCVAu7Tk5ftgcjv0jJlII2o7b8BqZc3sfEdwn1R6Ic7" # API secret
access_token = "821415961467228161-iB85g0Lm8c4jLqIqxWcryWjE8nm6CPq"
access_secret = "BrNaqN0BP2K3rYzIurlaTIaJeOk4MBP6mzBtR73ay5ulU"

In [2]:
auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
user_list = ["fotomac"] # Screen names

In [4]:
# Cleans tweets' text
def cleans_tweet_text(text):
    # Cleans Emoji
    emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)
    # Deletes emoji
    text = emoji_pattern.sub(r'', text)
    # Deletes urls
    result_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', r'', text.encode('utf-8'))
    return result_text

### Method 1:

In [5]:
for screen_name in user_list:
    try:
        fName = screen_name + '.txt' # To store the tweets in a text file. 
        with open(fName, 'w') as f:
            for tweet in tweepy.Cursor(api.user_timeline, screen_name = screen_name, count=200).items():
                text = cleans_tweet_text(tweet.text)
                print text
                f.write(text)
    except tweepy.TweepError as e:
        print(str(e.response))



Trabzonspor'da her şey 'OKAY' 
6 puan değerinde galibiyet  
Hugo Rodallega 
CANLI | 81' Trabzonspor 1-0 Karabükspor Gol: Okay Yokuşlu  
GOL: Trabzonspor-Karabükspor 1-0 (55dk Okay Yokuslu)
(İY): Trabzonspor-Karabükspor 0-0 
İşte o videonun 'bip'siz hali! 
Toroğlu küfür etti mi?  
İşte Beşiktaş'ın listesindeki 5 Brezilyalı isim  
Trabzonspor'un şampiyon kadrosunda yer alan efsane futbolcunun yeni mesleği... 
Bogdanovic Sacramento Kings'e mi gidiyor?  
PSV, Bruma için 5 milyon euro önermiş, G.Saray reddetmişti.. 

Tolgay iddiayı kazandı Babel'den 100 lirayı kaptı  
Bursaspor kafilesi otobüsüne yapılan saldırıyla ilgili olarak ifade  veren 12 kişi, çıkarıldıkları mahkemede serbes… 
Maliye Bakanı Naci Ağbal: "Eskişehirspor futbola renk ve değer katan bir kulüp. Sloganı ve futboluyla herkesin hafı… 
Fernando Torres: "Sağlık durumum için destek mesajı yollayanlara çok teşekkür ederim. Umarım çok yakın zamanda saha… 
Murat Cavcav, babasını temsil etmek için elinden geleni yapacağını belirtt

### Method 2:

In [6]:
for screen_name in user_list:
    fName = screen_name + '.txt' # To store the tweets in a text file. 
    with open(fName, 'w') as f:
        maxTweets = 10000000 # Pick as large as possible
        tweetsPerQry = 200  # The max the API permits
        sinceId = None # Each iteration we will update since id according to last fetched tweet
        max_id = -1L
        tweetCount = 0
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.user_timeline(screen_name = screen_name, count=tweetsPerQry)
                    else:
                        new_tweets = api.user_timeline(screen_name = screen_name, count=tweetsPerQry, since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = api.user_timeline(screen_name = screen_name, count=tweetsPerQry, max_id=str(max_id - 1))
                    else:
                        new_tweets = api.user_timeline(screen_name = screen_name, count=tweetsPerQry, max_id=str(max_id - 1), since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                for tweet in new_tweets:
                    maxTweets = tweet.author.statuses_count
                    text = cleans_tweet_text(tweet.text)
                    f.write(text)   
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id
            except tweepy.TweepError as e:
                # Just exit if any error
                print("some error : " + str(e))
                break

Downloaded 200 tweets
Downloaded 400 tweets
Downloaded 600 tweets
Downloaded 800 tweets
Downloaded 1000 tweets
Downloaded 1200 tweets
Downloaded 1400 tweets
Downloaded 1600 tweets
Downloaded 1800 tweets
Downloaded 2000 tweets
Downloaded 2200 tweets
Downloaded 2400 tweets
Downloaded 2600 tweets
Downloaded 2800 tweets
Downloaded 3000 tweets
Downloaded 3200 tweets
Downloaded 3248 tweets
No more tweets found
